In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os
import json
from IPython.display import clear_output
import itertools

# Concat dataset - pos early & late / neg early & late / neu early & late / total

* positive early dataset

In [ ]:
directory = '/content/drive/MyDrive/covid analysis/positive_1'
pos_early = pd.DataFrame(columns=['text', 'score', 'index'])

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    data = pd.read_csv(directory + '/' +filename, header=None, names=['index', 'text', 'ave_sentiment'])
    pos_early = pd.concat([pos_early, data])

print("Data has been merged")
pos_early = pos_early.astype({'text': 'str'})

pos_early = pos_early.drop(['index', 'score', 'ave_sentiment'], axis = 1)
print(pos_early.head())

Data has been merged
                                                text
0                                                 V1
1  ['surgeon', 'gener', 'look', 'kyli', 'jenner',...
2  ['hawaiian', 'airlin', 'flight', 'attend', 'te...
3  ['break', 'news', 'give', 'review', 'nation', ...
4  ['lo', 'angel', 'counti', 'report', 'two', 'de...


In [ ]:
data = []
for i in range(len(pos_early)):
  if pos_early.iloc[i][0] != 'V1' :
    data.append(pos_early.iloc[i][0])

In [ ]:
pos_er = []
stopwords = ['u', 'dont', 'cant', 'th', 'gt', 'pm', 'im', 'bc', 'dr', 'oct', 'may', 'octob',
             'plea', 'like', 'thank', 'get', 'could', 'say', 'jab', 'even', 'go', 'use', 'need',
             'work', 'thing', 'make', 'give', 'know', 'set', 'want']
for i in data :
  doc = ''
  for j in i.split() :
    word = ''.join(filter(str.isalnum, j))
    if word not in stopwords:
      doc += word + ' '
  pos_er.append(doc)

* LDA with Gensim

In [ ]:
!pip install pyLDAvis
!pip install spacy-langdetect
!pip install language-detector
!pip install symspellpy
!pip install sentence-transformers
clear_output()

In [ ]:
!pip install umap

  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3565 sha256=a1108af7c24025e9a471d0d285fcab5c34f71521eca3fa5b6a13d86e10f54b81
  Stored in directory: /root/.cache/pip/wheels/65/55/85/945cfb3d67373767e4dc3e9629300a926edde52633df4f0efe
Successfully built umap


In [ ]:
from tqdm import tgrange
import gensim
import gensim.corpora as corpora
from pprint import pprint
from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import warnings
import time
import datetime
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import umap
from collections import Counter
from sklearn.metrics import silhouette_score
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
pos_er = np.array(pos_er)
pe = [d.split() for d in pos_er]
dict_pe = corpora.Dictionary(pe)
dtm_pe = [dict_pe.doc2bow(i) for i in pe]

In [ ]:
print(pos_er[:5])

['surgeon gener look kyli jenner influenc address absolut seriou '
 'hawaiian airlin flight attend test posit kitv honolulu '
 'break news review nation park wish '
 'lo angel counti report two death new case mar et '
 'myth myth fact jantacurfew ']


* k = 6

In [ ]:
num = 6
LDA_pe = gensim.models.ldamodel.LdaModel(corpus = dtm_pe, id2word = dict_pe, num_topics = num, random_state = 100)
pprint(LDA_pe.print_topics())
doc_pe = LDA_pe[dtm_pe]

[(0,
  '0.066*"test" + 0.047*"posit" + 0.022*"respons" + 0.014*"news" + '
  '0.010*"first" + 0.009*"two" + 0.009*"patient" + 0.006*"here" + '
  '0.006*"question" + 0.005*"die"'),
 (1,
  '0.036*"trump" + 0.022*"new" + 0.011*"come" + 0.008*"post" + 0.007*"world" + '
  '0.007*"american" + 0.007*"vaccin" + 0.006*"money" + 0.006*"drug" + '
  '0.006*"news"'),
 (2,
  '0.075*"new" + 0.069*"case" + 0.035*"death" + 0.023*"confirm" + 0.019*"york" '
  '+ 0.019*"updat" + 0.019*"total" + 0.019*"home" + 0.017*"stay" + '
  '0.017*"report"'),
 (3,
  '0.017*"peopl" + 0.013*"good" + 0.010*"time" + 0.009*"hope" + 0.009*"well" + '
  '0.009*"right" + 0.009*"cure" + 0.008*"great" + 0.008*"take" + 0.007*"stop"'),
 (4,
  '0.018*"care" + 0.014*"support" + 0.013*"help" + 0.013*"relief" + '
  '0.013*"health" + 0.011*"donat" + 0.010*"april" + 0.010*"worker" + '
  '0.010*"protect" + 0.009*"patient"'),
 (5,
  '0.023*"mask" + 0.018*"fight" + 0.013*"keep" + 0.012*"face" + 0.012*"share" '
  '+ 0.012*"spread" + 0.010*"s

In [ ]:
cm = CoherenceModel(model = LDA_pe, texts = pe, corpus = dtm_pe, dictionary = dict_pe, coherence='c_v')
coherence = cm.get_coherence()
print("Coherence : ", coherence)

umass = CoherenceModel(model = LDA_pe, corpus = dtm_pe, coherence = 'u_mass')
coherence_um = umass.get_coherence()
print("Coherence : ", coherence_um)

Coherence :  0.26074957033563345
Coherence :  -4.42865130767207


In [ ]:
perplex = LDA_pe.log_perplexity(dtm_pe)
print("Perplexity : ", perplex)

Perplexity :  -8.617766108679959


In [ ]:
pyldavis_html_path = 'positive_early_lda_6.html'
data_pe = gensimvis.prepare(LDA_pe, dtm_pe, dict_pe)
pyLDAvis.display(data_pe)
pyLDAvis.save_html(data_pe, pyldavis_html_path)

* Find the best number of topics

In [ ]:
# 최적의 토픽 수 구하기
coherences_cv=[]
coherences_um=[]
perplexitiesT=[]
passes=[]
warnings.filterwarnings('ignore')

for i in range(10):
    if i==0:
        ntopics = 2
    else:
        ntopics += 2
    #nwords = 100
    tic = time.time()
    lda4 = gensim.models.ldamodel.LdaModel(corpus = dtm_pe, id2word=dict_pe, num_topics=ntopics)
    print('ntopics',ntopics,time.time() - tic)

    cm_cv = CoherenceModel(model=lda4, texts = pe, corpus = dtm_pe, dictionary = dict_pe, coherence='c_v')
    coherence_cv = cm_cv.get_coherence()
    print("Coherence_cv",coherence_cv)
    coherences_cv.append(coherence_cv)

    umass = CoherenceModel(model = lda4, corpus = dtm_pe, coherence = 'u_mass')
    coherence_um = umass.get_coherence()
    print("Coherence_um",coherence_um)
    coherences_um.append(coherence_um)

    perpl = lda4.log_perplexity(dtm_pe)
    print('Perplexity: ', perpl,'\n\n')
    perplexitiesT.append(perpl)

ntopics 2 219.44098687171936
Coherence_cv 0.17699770458179998
Coherence_um -3.5775045194335995
Perplexity:  -8.238382574238553 


ntopics 4 256.4297847747803
Coherence_cv 0.27268609563272533
Coherence_um -3.8298262337750426
Perplexity:  -8.366994215260481 


ntopics 6 258.87941312789917
Coherence_cv 0.29285359397341465
Coherence_um -4.237019337154708
Perplexity:  -8.498822530844459 


ntopics 8 284.9136538505554
Coherence_cv 0.25793526667730876
Coherence_um -4.497609476014244
Perplexity:  -8.64415356545989 


ntopics 10 299.7884361743927
Coherence_cv 0.2768453580041545
Coherence_um -4.513394726474841
Perplexity:  -8.728299634917606 


ntopics 12 310.89556217193604
Coherence_cv 0.266738185095413
Coherence_um -4.702301886472412
Perplexity:  -8.813176318727729 


ntopics 14 323.8230531215668
Coherence_cv 0.2838497808017101
Coherence_um -4.913954268138489
Perplexity:  -8.893570375276619 


ntopics 16 338.0452733039856
Coherence_cv 0.25833581518427934
Coherence_um -5.102633602049363
Perplex

* positive late dataset

In [ ]:
directory = '/content/drive/MyDrive/covid analysis/positive_2'
pos_late = pd.DataFrame(columns=['text', 'score', 'index'])

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    data = pd.read_csv(directory + '/' +filename, header=None, names=['index', 'text', 'ave_sentiment'])
    pos_late = pd.concat([pos_late, data])

print("Data has been merged")
pos_late = pos_late.astype({'text': 'str'})

pos_late = pos_late.drop(['index', 'score', 'ave_sentiment'], axis = 1)
print(pos_late.head())

Data has been merged
                                                text
0                                                 V1
1        ['emmi', 'grant', 'exempt', 'mask', 'rule']
2  ['chief', 'dr', 'tedro', 'welcom', 'india', 'c...
3  ['capo', 'talk', 'dalla', 'cultur', 'relations...
4  ['imf', 'call', 'coordin', 'action', 'account'...


In [ ]:
data = []
for i in range(len(pos_late)):
  if pos_late.iloc[i][0] != 'V1' :
    data.append(pos_late.iloc[i][0])
pos_la = []
for i in data :
  doc = ''
  for j in i.split() :
    word = ''.join(filter(str.isalnum, j))
    if word not in stopwords:
      doc += word + ' '
  pos_la.append(doc)

In [ ]:
pos_la = np.array(pos_la)
pl = [d.split() for d in pos_la]
dict_pl = corpora.Dictionary(pl)
dtm_pl = [dict_pl.doc2bow(i) for i in pl]

* k = 6

In [ ]:
num = 6
LDA_la = gensim.models.ldamodel.LdaModel(corpus = dtm_pl, id2word = dict_pl, num_topics = num, random_state = 100)
pprint(LDA_la.print_topics())
doc_la = LDA_la[dtm_pl]

[(0,
  '0.114*"new" + 0.101*"case" + 0.049*"death" + 0.038*"report" + 0.017*"updat" '
  '+ 0.013*"record" + 0.013*"latest" + 0.013*"daili" + 0.012*"confirm" + '
  '0.012*"activ"'),
 (1,
  '0.089*"test" + 0.040*"posit" + 0.021*"rapid" + 0.013*"unit" + '
  '0.013*"author" + 0.012*"doubl" + 0.011*"week" + 0.010*"result" + '
  '0.009*"clinic" + 0.009*"pfizerbiontech"'),
 (2,
  '0.044*"vaccin" + 0.015*"die" + 0.014*"new" + 0.011*"mandat" + '
  '0.009*"prevent" + 0.009*"peopl" + 0.009*"seek" + 0.008*"school" + '
  '0.008*"live" + 0.007*"york"'),
 (3,
  '0.037*"vaccin" + 0.028*"effect" + 0.027*"studi" + 0.015*"show" + '
  '0.015*"johnson" + 0.012*"month" + 0.011*"immun" + 0.009*"support" + '
  '0.008*"protect" + 0.008*"prevent"'),
 (4,
  '0.126*"vaccin" + 0.027*"booster" + 0.021*"pfizer" + 0.021*"dose" + '
  '0.017*"shot" + 0.015*"receiv" + 0.015*"approv" + 0.013*"fda" + '
  '0.013*"fulli" + 0.010*"million"'),
 (5,
  '0.015*"job" + 0.012*"impact" + 0.009*"plan" + 0.008*"emmi" + 0.008*"relief"

In [ ]:
warnings.filterwarnings('ignore')

cm = CoherenceModel(model = LDA_la, texts = pl, corpus = dtm_pl, dictionary = dict_pl, coherence='c_v')
coherence = cm.get_coherence()
print("Coherence : ", coherence)

umass = CoherenceModel(model = LDA_la, corpus = dtm_pl, coherence = 'u_mass')
coherence_um = umass.get_coherence()
print("Coherence : ", coherence_um)

perplex = LDA_la.log_perplexity(dtm_pl)
print("Perplexity : ", perplex)

Coherence :  0.3549873813397406
Coherence :  -5.3269806032228315
Perplexity :  -8.084753707425325


In [ ]:
pyldavis_html_path = 'positive_late_lda_6.html'
data_pl = gensimvis.prepare(LDA_la, dtm_pl, dict_pl)
pyLDAvis.display(data_pl)
pyLDAvis.save_html(data_pl, pyldavis_html_path)

In [ ]:
# 최적의 토픽 수 구하기
coherences_cv=[]
coherences_um=[]
perplexitiesT=[]
passes=[]
warnings.filterwarnings('ignore')

for i in range(10):
    if i==0:
        ntopics = 2
    else:
        ntopics += 2
    tic = time.time()
    lda4 = gensim.models.ldamodel.LdaModel(corpus = dtm_pl, id2word=dict_pl, num_topics=ntopics)
    print('ntopics',ntopics,time.time() - tic)

    cm_cv = CoherenceModel(model=lda4, texts = pl, corpus = dtm_pl, dictionary = dict_pl, coherence='c_v')
    coherence_cv = cm_cv.get_coherence()
    print("Coherence_cv",coherence_cv)
    coherences_cv.append(coherence_cv)

    umass = CoherenceModel(model = lda4, corpus = dtm_pl, coherence = 'u_mass')
    coherence_um = umass.get_coherence()
    print("Coherence_um",coherence_um)
    coherences_um.append(coherence_um)

    perpl = lda4.log_perplexity(dtm_pl)
    print('Perplexity: ', perpl,'\n\n')
    perplexitiesT.append(perpl)

ntopics 2 55.428255796432495
Coherence_cv 0.17319459583721747
Coherence_um -3.8912695338864953
Perplexity:  -7.826517364377903 


ntopics 4 64.19022178649902
Coherence_cv 0.21013171331323724
Coherence_um -4.965638247303935
Perplexity:  -7.952663200952096 


ntopics 6 63.31238079071045
Coherence_cv 0.34980586691389665
Coherence_um -5.281791222514632
Perplexity:  -8.04408097702642 


ntopics 8 63.649758100509644
Coherence_cv 0.3201856752012163
Coherence_um -5.486369668572315
Perplexity:  -8.133907976158424 


ntopics 10 65.25340008735657
Coherence_cv 0.32807093243264446
Coherence_um -5.7974985485968515
Perplexity:  -8.210272539349809 


ntopics 12 66.95155715942383
Coherence_cv 0.32375722526518247
Coherence_um -6.922519539334515
Perplexity:  -8.300924701306464 


ntopics 14 68.70925426483154
Coherence_cv 0.33758238702449533
Coherence_um -6.7401953431123545
Perplexity:  -8.367202468016638 


ntopics 16 72.97786951065063
Coherence_cv 0.3136299374743159
Coherence_um -6.334421299073037
Perpl

# LDA + BERT

In [ ]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from collections import Counter
from sklearn.metrics import silhouette_score
import umap
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
import os

In [ ]:
def get_topic_words(token_lists, labels, k=None):
    """
    get top words within each topic from clustering results
    """
    if k is None:
        k = len(np.unique(labels))

    topics = ['' for _ in range(k)]

    for i, c in enumerate(token_lists):
        topics[labels[i]] += (' ' + ' '.join(c))
        if i%100000 == 0 :
          print(i)

    word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
    # get topics
    topics = list(map(lambda x: list(map(lambda x: x[0], x[:10])), word_counts))

    return topics

def get_coherence(model, token_lists, measure='c_v'):
    """
    Get model coherence from gensim.models.coherencemodel
    :param model: Topic_Model object
    :param token_lists: token lists of docs
    :param topics: topics as top words
    :param measure: coherence metrics
    :return: coherence score
    """
    if model.method == 'LDA':
        cm = CoherenceModel(model=model.ldamodel, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    else:
        print("Start get topic words")
        topics = get_topic_words(token_lists, model.cluster_model.labels_)
        print("Done! and start get coherence")
        cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
        print("Get coherence done!")
    return cm.get_coherence()

def get_silhouette(model):
    """
    Get silhouette score from model
    :param model: Topic_Model object
    :return: silhouette score
    """
    if model.method == 'LDA':
        return
    lbs = model.cluster_model.labels_
    vec = model.vec[model.method]
    return silhouette_score(vec, lbs)

def plot_proj(embedding, lbs):
    """
    Plot UMAP embeddings
    :param embedding: UMAP (or other) embeddings
    :param lbs: labels
    """
    n = len(embedding)
    counter = Counter(lbs)
    for i in range(len(np.unique(lbs))):
        plt.plot(embedding[:, 0][lbs == i], embedding[:, 1][lbs == i], '.', alpha=0.5,
                 label='cluster {}: {:.2f}%'.format(i, counter[i] / n * 100))
    plt.legend(loc = 'best')
    plt.grid(color ='grey', linestyle='-',linewidth = 0.25)


def visualize(model):
    """
    Visualize the result for the topic model by 2D embedding (UMAP)
    :param model: Topic_Model object
    """
    if model.method == 'LDA':
        return
    reducer = umap.UMAP()
    print('Calculating UMAP projection ...')
    vec_umap = reducer.fit_transform(model.vec[model.method])
    print('Calculating UMAP projection. Done!')
    plot_proj(vec_umap, model.cluster_model.labels_)
    dr = '/content/drive/MyDrive/covid analysis/lda+bert result/{}/{}'.format(model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig('/2D_vis')

def get_wordcloud(model, token_lists, topic):
    """
    Get word cloud of each topic from fitted model
    :param model: Topic_Model object
    :param sentences: preprocessed sentences from docs
    """
    if model.method == 'LDA':
        return
    print('Getting wordcloud for topic {} ...'.format(topic))
    lbs = model.cluster_model.labels_
    tokens = ' '.join([' '.join(_) for _ in np.array(token_lists)[lbs == topic]])

    wordcloud = WordCloud(width=800, height=560,
                          background_color='white', collocations=False,
                          min_font_size=10).generate(tokens)

    # plot the WordCloud image
    plt.figure(figsize=(8, 5.6), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    dr = '/content/drive/MyDrive/covid analysis/lda+bert result/{}/{}'.format(model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig('/content/drive/MyDrive/covid analysis/lda+bert result' + '/Topic' + str(topic) + '_wordcloud')
    print('Getting wordcloud for topic {}. Done!'.format(topic))


In [ ]:
class Autoencoder:
    """
    Autoencoder for learning latent space representation
    architecture simplified for only one hidden layer
    """

    def __init__(self, latent_dim=32, activation='relu', epochs=200, batch_size =128):
        self.latent_dim = latent_dim
        self.activation = activation
        self.epochs = epochs
        self.batch_size = batch_size
        self.autoencoder = None
        self.encoder = None
        self.decoder = None
        self.his = None

    def _compile(self, input_dim):
        """
        compile the computational graph
        """
        input_vec = Input(shape=(input_dim,))
        encoded = Dense(self.latent_dim, activation=self.activation)(input_vec)
        decoded = Dense(input_dim, activation=self.activation)(encoded)
        self.autoencoder = Model(input_vec, decoded)
        self.encoder = Model(input_vec, encoded)
        encoded_input = Input(shape=(self.latent_dim,))
        decoder_layer = self.autoencoder.layers[-1]
        self.decoder = Model(encoded_input, self.autoencoder.layers[-1](encoded_input))
        self.autoencoder.compile(optimizer='adam', loss=keras.losses.mean_squared_error)

    def fit(self, X):
        if not self.autoencoder:
            self._compile(X.shape[1])
        X_train, X_test = train_test_split(X)
        self.his = self.autoencoder.fit(X_train, X_train,
                                        epochs=200,
                                        batch_size=128,
                                        shuffle=True,
                                        validation_data=(X_test, X_test), verbose=0)

# Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim import corpora
import gensim
import numpy as np
#from Autoencoder import *
#from preprocess import *
from datetime import datetime

In [ ]:
# define model object
class Topic_Model:
    def __init__(self, k=6, method='TFIDF'):
        """
        :param k: number of topics
        :param method: method chosen for the topic model
        """
        if method not in {'TFIDF', 'LDA', 'BERT', 'LDA_BERT'}:
            raise Exception('Invalid method!')

        self.k = k
        self.dictionary = None
        self.corpus = None
        self.cluster_model = None
        self.ldamodel = None
        self.vec = {}
        self.gamma = 15  # parameter for reletive importance of lda
        self.method = method
        self.AE = None
        self.id = method + '_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

    def vectorize(self, sentences, token_lists, method=None):
        """
        Get vecotr representations from selected methods
        """
        # Default method
        if method is None:
            method = self.method

        # turn tokenized documents into a id <-> term dictionary
        self.dictionary = corpora.Dictionary(token_lists)
        # convert tokenized documents into a document-term matrix
        self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        if method == 'TFIDF':
            print('Getting vector representations for TF-IDF ...')
            tfidf = TfidfVectorizer()
            vec = tfidf.fit_transform(sentences)
            print('Getting vector representations for TF-IDF. Done!')
            return vec

        elif method == 'LDA':
            print('Getting vector representations for LDA ...')
            if not self.ldamodel:
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary)

            def get_vec_lda(model, corpus, k):
                """
                Get the LDA vector representation (probabilistic topic assignments for all documents)
                :return: vec_lda with dimension: (n_doc * n_topic)
                """
                n_doc = len(corpus)
                vec_lda = np.zeros((n_doc, k))
                for i in range(n_doc):
                    # get the distribution for the i-th document in corpus
                    for topic, prob in model.get_document_topics(corpus[i]):
                        vec_lda[i, topic] = prob

                return vec_lda

            vec = get_vec_lda(self.ldamodel, self.corpus, self.k)
            print('Getting vector representations for LDA. Done!')
            return vec

        elif method == 'BERT':

            print('Getting vector representations for BERT ...')
            from sentence_transformers import SentenceTransformer
            model = SentenceTransformer('bert-base-nli-max-tokens')
            vec = np.array(model.encode(sentences, show_progress_bar=True))
            print('Getting vector representations for BERT. Done!')
            return vec

             
        elif method == 'LDA_BERT':
        #else:
            vec_lda = self.vectorize(sentences, token_lists, method='LDA')
            vec_bert = self.vectorize(sentences, token_lists, method='BERT')
            vec_ldabert = np.c_[vec_lda * self.gamma, vec_bert]
            self.vec['LDA_BERT_FULL'] = vec_ldabert
            if not self.AE:
                self.AE = Autoencoder()
                print('Fitting Autoencoder ...')
                self.AE.fit(vec_ldabert)
                print('Fitting Autoencoder Done!')
            vec = self.AE.encoder.predict(vec_ldabert)
            return vec

    def fit(self, sentences, token_lists, method=None, m_clustering=None):
        """
        Fit the topic model for selected method given the preprocessed data
        :docs: list of documents, each doc is preprocessed as tokens
        :return:
        """
        # Default method
        if method is None:
            method = self.method
        # Default clustering method
        if m_clustering is None:
            m_clustering = KMeans

        # turn tokenized documents into a id <-> term dictionary
        if not self.dictionary:
            self.dictionary = corpora.Dictionary(token_lists)
            # convert tokenized documents into a document-term matrix
            self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        ####################################################
        #### Getting ldamodel or vector representations ####
        ####################################################

        if method == 'LDA':
            if not self.ldamodel:
                print('Fitting LDA ...')
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary)
                print('Fitting LDA Done!')
        else:
            print('Clustering embeddings ...')
            self.cluster_model = m_clustering(self.k)
            self.vec[method] = self.vectorize(sentences, token_lists, method)
            self.cluster_model.fit(self.vec[method])
            print('Clustering embeddings. Done!')

    def predict(self, sentences, token_lists, out_of_sample=None):
        """
        Predict topics for new_documents
        """
        # Default as False
        out_of_sample = out_of_sample is not None

        if out_of_sample:
            corpus = [self.dictionary.doc2bow(text) for text in token_lists]
            if self.method != 'LDA':
                vec = self.vectorize(sentences, token_lists)
                print(vec)
        else:
            corpus = self.corpus
            vec = self.vec.get(self.method, None)

        if self.method == "LDA":
            lbs = np.array(list(map(lambda x: sorted(self.ldamodel.get_document_topics(x),
                                                     key=lambda x: x[1], reverse=True)[0][0],
                                    corpus)))
        else:
            lbs = self.cluster_model.predict(vec)
        return lbs

# Training

In [ ]:
import argparse
import pickle

def main(choice):
    
    method = choice
    print(method)
    samp_size = 51000
    ntopic = 6

    data = np.array(pos_er)
    pe = [d.split() for d in pos_er]
    dict_pe = corpora.Dictionary(pe)
    dtm_pe = [dict_pe.doc2bow(i) for i in pe]
    sentences, token_lists, idx_in = data, pe, dtm_pe

    # Define the topic model object
    tm = Topic_Model(k = ntopic, method = method)

    # Fit the topic model by chosen method
    tm.fit(sentences, token_lists)

    # Evaluate using metrics
    #with open("/content/drive/MyDrive/covid analysis/lda+bert result/{}.file".format(tm.id), "wb") as f:
    #    pickle.dump(tm, f, pickle.HIGHEST_PROTOCOL)

    print('Coherence:', get_coherence(tm, token_lists, 'c_v'))
    print('Silhouette Score:', get_silhouette(tm))
    # visualize and save img
    visualize(tm)
    for i in range(tm.k):
        get_wordcloud(tm, token_lists, i)

In [ ]:
choose = ['LDA', 'BERT', 'LDA_BERT']
for i in choose :
  main(i)

LDA
Fitting LDA ...
Fitting LDA Done!
Coherence: 0.2400824519893209
Silhouette Score: None
BERT
Clustering embeddings ...
Getting vector representations for BERT ...


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/397 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/34167 [00:00<?, ?it/s]